Because of the partial acquisitions, bbregister fails for some runs of some subjects. First shot at fixing it will be to re-run bbregister with the lyman registration outputs as an initial first guess


In [1]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil
import gc

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [21]:
#re-arrange directory structure
exp = 'mag'
coreg_files = ['func2anat.png','func2anat_tkreg.dat','func2anat_flirt.mat']
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        orig_coreg = sub_path + 'orig_coreg/'

        if os.path.exists(sub_path) and not os.path.exists(orig_coreg):
            os.mkdir(orig_coreg)

            for f in coreg_files:
                old = sub_path + f
                new = orig_coreg + f
                os.rename(old,new)

In [43]:
def rerun_coreg(in_tuple):
    sub, exp, run = in_tuple
    sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'

    if os.path.exists(sub_path):    
        #skull strip mean_func
        mov_file = sub_path + 'mean_func.nii.gz' 
        new_mov_file = sub_path + 'mean_func_bet.nii.gz' 
#         os.remove(new_mov_file)
        cmd = ['bet2',mov_file,new_mov_file,'-f','.05']
        cmd = ' '.join(cmd)
        if not os.path.exists(new_mov_file):
            os.system(cmd)
        
        #run bbregister
        fs_dir = home_dir +'/data/' + sub
        init_reg = sub_path +  'orig_coreg/func2anat_tkreg.dat'
        out_reg = sub_path + 'func2anat_tkreg.dat'
        out_reg_fsl = sub_path + 'func2anat_flirt.mat'
        out_coreg = sub_path + '/coreg_mean_func.nii.gz'
        cmd = ['bbregister','--mov',new_mov_file,'--s',
               fs_dir,'--bold','--init-reg', init_reg, 
               '--reg',out_reg, '--fslmat',out_reg_fsl,
               '--o',out_coreg]
        cmd = ' '.join(cmd)
        print cmd
        if not os.path.exists(out_coreg):
            os.system(cmd)

        #save output
        out_png = sub_path + "func2anat.png"
        wm_file = fs_dir + '/mri/wm.mgz'
        wm_data = nib.load(wm_file).get_data().astype(bool).astype(int)
#         m = Mosaic(out_coreg, wm_data, step=3)
#         m.plot_contours(["#DD2222"])
#         m.savefig(out_png)
#         m.close()
rerun_coreg(('subject14','mag',4))

bbregister --mov /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/mean_func_bet.nii.gz --s /data/home/iballard/mag//data/subject14 --bold --init-reg /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/orig_coreg/func2anat_tkreg.dat --reg /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/func2anat_tkreg.dat --fslmat /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/func2anat_flirt.mat --o /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4//coreg_mean_func.nii.gz


In [24]:
in_tuples = []
for exp in ['mag']:
    for sub in sub_list:
        for run in range(4,5):
            in_tuples.append((sub,exp,run))

In [25]:
pool = multiprocessing.Pool(processes = 20)
output = pool.map(rerun_coreg,in_tuples)
pool.terminate()
pool.join()

In [46]:
redo_tuples = []
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        f = sub_path + 'func2anat_tkreg.dat.mincost'
        if os.path.exists(f):
            cost = np.loadtxt(f)
            cost = cost[0]
            print cost
            if cost > .6:
                print sub, run
                redo_tuples.append((sub,exp,run))
        else:
            print 'nodata',sub,run

0.492148
0.490176
0.487275
0.484607
0.461239
0.463207
0.464501
0.462331
0.55356
0.553509
0.55518
0.558624
0.440556
0.442542
0.454576
0.445424
0.473904
0.477038
0.488446
0.498954
0.431487
0.437099
0.463472
nodata subject09 4
0.48515
0.487702
0.49193
0.491115
0.430224
0.43262
0.433923
0.435068
0.497191
0.507853
0.493062
0.501537
0.420219
0.423924
0.435528
0.441342
0.537177
0.561796
0.548525
0.553554
0.437964
0.436719
0.465622
0.436623
0.439644
0.442133
0.448892
0.449193
0.551406
0.549727
0.550997
0.55018


In [44]:
#make image after hand tweaking troublesome registrations
for in_tuple in redo_tuples:
    #save output
    sub, exp, run = in_tuple
    sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
    fs_dir = home_dir +'/data/' + sub
    out_coreg = sub_path + '/coreg_mean_func.nii.gz'
        
    out_png = sub_path + "func2anat.png"
    wm_file = fs_dir + '/mri/wm.mgz'
    wm_data = nib.load(wm_file).get_data().astype(bool).astype(int)
    m = Mosaic(out_coreg, wm_data, step=3)
    m.plot_contours(["#DD2222"])
    m.savefig(out_png)
    m.close()

In [53]:
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        if os.path.exists(sub_path):
            mask = sub_path + '/functional_mask.nii.gz'
            mean = sub_path + '/mean_func.nii.gz'
            cmd = ['fslcpgeom',mean,mask]
            cmd = ' '.join(cmd)
            os.system(cmd)